In [56]:
import pickle
from collections import defaultdict
import pandas as pd

In [57]:
with open('./feedback.pkl', 'rb') as f:
    feedback = pickle.load(f)

In [58]:
print(type(feedback))
relevant_fields = [
    'TargetQuaternion',
    'ActualQuaternion',
    'QTarget',
    'QActual',
    'QDTarget',
    'QDActual',
    'CurrentCommandId',
    'RobotMode',
]
for x in relevant_fields:
    print(x, feedback[x][0])

<class 'numpy.ndarray'>
TargetQuaternion [ 5.55111512e-17  5.55111512e-17  7.07106769e-01 -7.07106769e-01]
ActualQuaternion [ 4.48877017e-06  2.11853167e-05  7.07106769e-01 -7.07106769e-01]
QTarget [0. 0. 0. 0. 0. 0.]
QActual [ 2.03954114e-05  0.00000000e+00  2.03954114e-05  1.37329102e-03
 -1.37329102e-03  6.86645508e-04]
QDTarget [0. 0. 0. 0. 0. 0.]
QDActual [ 0.02730266 -0.02548747 -0.01919208  1.82304382 -0.79925537  3.13316345]
CurrentCommandId 4
RobotMode 5


read feedback data

In [59]:
def read_buffered_pickle(filename='./feedback_all.pkl'):
    all_data = []
    try:
        with open(filename, 'rb') as f:
            while True:
                batch = pickle.load(f)
                all_data.extend(batch)
    except EOFError:
        pass
    return all_data

feedback = read_buffered_pickle()
print(f"Total data points loaded: {len(feedback)}")

Total data points loaded: 4300


read (command id, path) pairs

In [60]:
with open('./command_path_mapping.pkl', 'rb') as f:
    command_path_mapping = pickle.load(f)

pandas

In [61]:
df = pd.DataFrame(feedback)

data wrangling

In [62]:
df['path_id'] = df['CurrentCommandId'].map(command_path_mapping)
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
df_sorted = df.sort_values(by=['path_id', 'TimeStamp'], ascending=True)

In [63]:
print(df_sorted)
df_sorted.to_csv('./feedback.csv')

                                       TargetQuaternion  \
91    [1.1102230246251565e-16, -1.0, -2.775557561562...   
92    [2.7755575615628914e-17, -1.0, -4.163336342344...   
93    [1.3877787807814457e-16, -1.0, -4.718447854656...   
94    [5.551115123125783e-17, -1.0, -3.8857805861880...   
95    [1.6653345369377348e-16, -1.0, -2.220446049250...   
...                                                 ...   
4295  [-2.498001805406602e-16, -1.0, -3.053113317719...   
4296  [-2.498001805406602e-16, -1.0, -1.665334536937...   
4297  [1.6653345369377348e-16, -1.0, 0.0, -5.5511151...   
4298  [-1.3877787807814457e-16, -1.0, -1.94289029309...   
4299  [-1.1102230246251565e-16, -1.0, 0.0, -8.326672...   

                                       ActualQuaternion  \
91    [-1.7342705177725293e-05, -1.0, 4.568132226268...   
92    [-6.50186257189489e-06, -1.0, -1.3645369108417...   
93    [-1.352712388325017e-05, -1.0, -1.352653725916...   
94    [-7.041078333713813e-06, -1.0, 4.3310042201483...